In [9]:
%reload_ext autoreload
%autoreload 2
from main import initialize
import time, pygame.midi
from musenet_api import decode_musenet, musenet_generate, encode_musenet
def reset_pygame():
    pygame.midi.quit()
    initialize()

reset_pygame()

Initialized. Device List:
(b'CoreMIDI', b'IAC Driver Bus 1', 1, 0, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 1, 0, 0)
(b'CoreMIDI', b'IAC Driver Bus 1', 0, 1, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 0, 1, 0)


## Setup libs for playing

In [36]:
import time
def setup_players():
    # Setup code for PLAYING
    try:
        pygame.midi.quit()
        pygame.midi.init()
        players = [
            pygame.midi.Output(2),
            pygame.midi.Output(3)
        ]
        for player in players:
            player.set_instrument(0)
    except Exception as e:
        print(e)
    return players

players = setup_players()

def play_note(note, velocity):
    for player in players:
        player.note_on(note, velocity,1)

def reset_note(note):
    for player in players:
        player.note_off(note, 100, 1)

def play_notes(notes, offset=0):
    played_notes = {}
    for note in notes:
        cur_time = time.time()
        print(note)
        if note["type"] == "wait":
            time.sleep(note["delay"]*0.00923081517)
        elif note["type"] == "invalid":
            break
        else:
            played_notes[note["pitch"]] = cur_time
            play_note(
                note["pitch"] - offset,
                note["volume"]
            )
        for note in list(played_notes.keys()):
            last_played_time = played_notes[note]
            if (cur_time - last_played_time) > 1:
                reset_note(note)

def reset_notes():
    for note in range(128):
        time.sleep(0.01)
        reset_note(note)

def play_musenet_response(
    completions,
    input,
    play_input=False,
    idx=0,
    offset=0
):
    encoded_input = encode_musenet(input)
    encoded_response = completions[idx]["encoding"]

    reset_notes()

    if not play_input:
        encoded_response = encoded_response[len(encoded_input):].strip()

    musenet_response = decode_musenet(
        encoded_response
    )
    play_notes(musenet_response, offset)


players

## Interactive

In [13]:
# Get input
from main import input_main
reset_pygame()
midi_notes = input_main(1, max_time=7500)
print("Recorded", len(midi_notes))

Initialized. Device List:
(b'CoreMIDI', b'IAC Driver Bus 1', 1, 0, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 1, 0, 0)
(b'CoreMIDI', b'IAC Driver Bus 1', 0, 1, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 0, 1, 0)
using input_id :1:
135192


KeyboardInterrupt: 

In [13]:
# Get response
res = musenet_generate(
    encode_musenet(midi_notes),
    length=60
)
completions = res.json()["completions"]

In [46]:
import time, pygame.midi

# Setup code for PLAYING
try:
    pygame.midi.init()
    players = [
        pygame.midi.Output(2),
        pygame.midi.Output(3)
    ]
    for player in players:
        player.set_instrument(0)
except:
    pass

play_musenet_response(
    completions, 
    midi_notes,
    play_input=True
)

## Loop

In [15]:
reset_pygame()
players = setup_players()

for i in range(128):
    play_note(i, 100)
    time.sleep(0.011)
    for player in players:
        player.note_off(i, 100, 1)

Initialized. Device List:
(b'CoreMIDI', b'IAC Driver Bus 1', 1, 0, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 1, 0, 0)
(b'CoreMIDI', b'IAC Driver Bus 1', 0, 1, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 0, 1, 0)


In [39]:
import time, pygame.midi
from main import input_main

reset_notes()

while True:
    reset_pygame()
    print("GETTING INPUT")
    midi_notes = input_main(1, max_time=7500)
    players = setup_players()
    reset_notes()
    if len(midi_notes) > 0:
        print("Getting reponse...")
        start_time = time.time()
        # Get response
        res = musenet_generate(
            encode_musenet(midi_notes),
            length=70
        )
        print(f"Took {time.time() - start_time:.2f}s")
        completions = res.json()["completions"]

        play_musenet_response(
            completions, 
            midi_notes,
            offset=+16
        )

        for i in range(128):
            time.sleep(0.011)
            for player in players:
                player.note_off(i, 100, 1)
    else:
        print("Did not play, empty input")

Initialized. Device List:
(b'CoreMIDI', b'IAC Driver Bus 1', 1, 0, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 1, 0, 0)
(b'CoreMIDI', b'IAC Driver Bus 1', 0, 1, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 0, 1, 0)
GETTING INPUT
using input_id :1:
808380
Did not play, empty input
Initialized. Device List:
(b'CoreMIDI', b'IAC Driver Bus 1', 1, 0, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 1, 0, 0)
(b'CoreMIDI', b'IAC Driver Bus 1', 0, 1, 0)
(b'CoreMIDI', b'Alesis Recital Pro ', 0, 1, 0)
GETTING INPUT
using input_id :1:
809805
Time is up
Getting reponse...
Took 8.45s
{'type': 'wait', 'delay': 49}
{'type': 'note', 'pitch': 64, 'instrument': 'piano', 'volume': 72}
{'type': 'wait', 'delay': 54}
{'type': 'note', 'pitch': 72, 'instrument': 'piano', 'volume': 48}
{'type': 'wait', 'delay': 2}
{'type': 'note', 'pitch': 64, 'instrument': 'piano', 'volume': 64}
{'type': 'wait', 'delay': 1}
{'type': 'note', 'pitch': 76, 'instrument': 'piano', 'volume': 56}
{'type': 'wait', 'delay': 1}
{'type': 'note', 'pitc

KeyboardInterrupt: 

In [21]:
completions[0]

{'encoding': '1608 4108 560 3970 1205 3972 1337 4039 1340 4006 1344 4008 1347 4046 1212 3968 1216 4049 816 3968 565 3968 825 3968 1351 3970 1482 3972 845 4037 1351 4006 1482 4044 1095 4006 1610 4047 572 3968 705 3968 579 3968 970 3968 1101 3970 576 3968 972 3970 447 3968 463 3968 465 4032 1224 3971 1357 3973 465 4027 1359 3972 1236 3973 1238 3975 1369 4031 1238 3974 1241 3975 477 4020 1369 3973 1373 3975 1376 4031 1243 3975 991 3976 1504 4034 1117 3979 1120 3971 1250 3977 1253 4042 1127 3992 875 4097 4228 4189 4096 3876 3897 2333 2613 2625 3968 2620 2629 3968 2617 3968 2620 3968 2625 3972 2741 3972 2333 2613 3978 2741 3970 3880',
 'humanReadableEncoding': 'piano:v112:C4 guitar_winds piano:v48:C2 wait:3 piano:v88:F2 wait:5 piano:v96:A3 wait:72 piano:v96:C3 wait:39 piano:v96:E3 wait:41 piano:v96:G3 wait:79 piano:v88:C3 wait:1 piano:v88:E3 wait:82 piano:v64:C2 wait:1 piano:v48:F2 wait:1 piano:v64:A3 wait:1 piano:v96:B4 wait:3 piano:v104:D4 wait:5 piano:v64:F4 wait:70 piano:v96:B4 wait:39 